# How to use OGC Moving Features CSV with Python

## To read Moving Features CSV data files
Python provides CSV package to read CSV data. A file encoded with Moving Features CSV can be read easily with the package. The following function reads OGC Moving Features CSV and returns tuple of a metadata dictionary and pandas.DataFrame.

In [1]:
import csv
import pandas
import numpy
import datetime

def read_mfcsv(filename):
    metadata = {}
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        stboundedby = next(reader)
        metadata['stboundedby'] = {
            'coordinate' : stboundedby[1:3],
            'bounds' : stboundedby[3:5],
            'timerange' : 
            {
               'starttime': datetime.datetime.strptime(stboundedby[5],'%Y-%m-%dT%H:%M:%Sz'),
               'endtime': datetime.datetime.strptime(stboundedby[6],'%Y-%m-%dT%H:%M:%Sz'),
               'unit': stboundedby[7]
            }
        }
        columninfo = next(reader)
        metadata['columninfo'] = {
            'attr_setting': dict(zip(columninfo[3::2],columninfo[4::2]))
        }
    databody = pandas.read_csv(filename, skiprows=2, header=None)
    databody.columns=['mfidref','starttime','endtime','trajectory']+list(metadata['columninfo']['attr_setting'].keys())
    databody['trajectory'] = databody['trajectory'].apply(lambda x: [[float(x),float(y)] for x,y in zip(x.split(' ')[0::2],x.split(' ')[1::2])])
    return metadata, databody

A sample file 'test_mf.csv' can be read as follows:

In [2]:
metadata, databody = read_mfcsv('mf.csv')
databody

,mfidref,starttime,endtime,trajectory,state,type code
0,a,10,120,"[[35.6815, 139.7651], [35.682, 139.7661]]",walking,1
1,b,10,190,"[[35.6811, 139.7662], [35.6818, 139.7661]]",walking,2
2,a,120,150,"[[35.682, 139.7661], [35.6834, 139.7662]]",walking,2
3,a,150,190,"[[35.6834, 139.7662], [35.6835, 139.7663]]",walking,2


The startime and the endtime can be easily converted into absolute style.

In [3]:
time_converter ={
     'sec' : lambda s,o: datetime.timedelta(seconds=s)+o,
     'minute' : lambda s,o: datetime.timedelta(minutes=s)+o,
     'absolute' : lambda s,o: s
}

origin = metadata['stboundedby']['timerange']['starttime']
unit = metadata['stboundedby']['timerange']['unit']

databody['starttime']=databody['starttime'].apply(lambda x: time_converter[unit](x,origin))
databody['endtime']=databody['endtime'].apply(lambda x: time_converter[unit](x,origin))
databody

,mfidref,starttime,endtime,trajectory,state,type code
0,a,2012-01-17 12:33:51,2012-01-17 12:35:41,"[[35.6815, 139.7651], [35.682, 139.7661]]",walking,1
1,b,2012-01-17 12:33:51,2012-01-17 12:36:51,"[[35.6811, 139.7662], [35.6818, 139.7661]]",walking,2
2,a,2012-01-17 12:35:41,2012-01-17 12:36:11,"[[35.682, 139.7661], [35.6834, 139.7662]]",walking,2
3,a,2012-01-17 12:36:11,2012-01-17 12:36:51,"[[35.6834, 139.7662], [35.6835, 139.7663]]",walking,2


## Plot the loaded tracks
folium is useful package for plotting geospatial data.

In [4]:
import folium
m = folium.Map(location=[35.6815,139.7651], zoom_start=17)
m

The list in column 'trajectory' should be added to the map for the visualization.

In [5]:
for l in databody['trajectory']:
    folium.PolyLine(locations=l).add_to(m)
m